In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.3")\
    .getOrCreate()


In [0]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

    # Start Spark Session with Spark NLP
spark = sparknlp.start()

    # Download a pre-trained pipeline
pipeline = PretrainedPipeline('explain_document_dl', lang='en')

    # Annotate your testing dataset
result = pipeline.annotate("The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris.")

  

explain_document_dl download started this may take some time.
Approx size to download 169.3 MB
[OK!]


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2810309188642085> in <module>
      8 
      9     # Download a pre-trained pipeline
---> 10 pipeline = PretrainedPipeline('explain_document_dl', lang='en')
     11 
     12     # Annotate your testing dataset

/databricks/python/lib/python3.8/site-packages/sparknlp/pretrained.py in __init__(self, name, lang, remote_loc, parse_embeddings, disk_location)
    142         else:
    143             self.model = PipelineModel.load(disk_location)
--> 144         self.light_model = LightPipeline(self.model, parse_embeddings)
    145 
    146     @staticmethod

/databricks/python/lib/python3.8/site-packages/sparknlp/base.py in __init__(self, pipelineModel, parse_embeddings)
     77     def __init__(self, pipelineModel, parse_embeddings=False):
     78         self.pipeline_model = pipelineModel
---> 79         self._lightPipeline = _in

In [0]:
%fs ls dbfs:/FileStore/tables/ApacheCourse/

path,name,size
dbfs:/FileStore/tables/ApacheCourse/co2_1800_1995.csv,co2_1800_1995.csv,56288
dbfs:/FileStore/tables/ApacheCourse/co2_1800_1995_Extended.csv,co2_1800_1995_Extended.csv,169870304
dbfs:/FileStore/tables/ApacheCourse/country_1.csv,country_1.csv,68387
dbfs:/FileStore/tables/ApacheCourse/dummy.csv,dummy.csv,79
dbfs:/FileStore/tables/ApacheCourse/dummy.parquet,dummy.parquet,3579
dbfs:/FileStore/tables/ApacheCourse/earthquake.csv,earthquake.csv,967203
dbfs:/FileStore/tables/ApacheCourse/reddit_worldnews.json,reddit_worldnews.json,252304


In [0]:
dataPath = "dbfs:/FileStore/tables/ApacheCourse/reddit_worldnews.json"
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)


In [0]:
title = "data.title"
author = "data.author"
df_Auth_title = df.select(title,author)

In [0]:
df_Auth_title.limit(10).toPandas()

,title,author
0,Microsoft Corp said it has discovered hacking ...,jaykirsch
1,Deutsche Bank reportedly planned to extend the...,canuck_burger
2,"Iranian ""morality police"" were forced to fire ...",honolulu_oahu_mod
3,Trump administration 'pushing Saudi nuclear de...,madam1
4,"NASA Happily Reports the Earth is Greener, Wit...",purplexxx
5,President Vladimir Putin threatened the United...,madam1
6,33 students at 2 Vancouver schools ordered to ...,littlebossman
7,Giant tortoises believed to have been extinct ...,joakinzz99
8,The use of marijuana for medical purposes in T...,cskarfors
9,Three Tory MPs join Labour breakaway group,carpie101


In [0]:
df_title = df_Auth_title.select('title').toDF("text")
df_title.printSchema()
df_title.limit(10).toPandas()

root
 |-- text: string (nullable = true)



,text
0,Microsoft Corp said it has discovered hacking ...
1,Deutsche Bank reportedly planned to extend the...
2,"Iranian ""morality police"" were forced to fire ..."
3,Trump administration 'pushing Saudi nuclear de...
4,"NASA Happily Reports the Earth is Greener, Wit..."
5,President Vladimir Putin threatened the United...
6,33 students at 2 Vancouver schools ordered to ...
7,Giant tortoises believed to have been extinct ...
8,The use of marijuana for medical purposes in T...
9,Three Tory MPs join Labour breakaway group


In [0]:
pipeline = PretrainedPipeline("explain_document_ml", lang="en")
df_annotation = pipeline.transform(df_title)


explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[OK!]


In [0]:
df_annotation.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|               spell|              lemmas|               stems|                 pos|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Microsoft Corp sa...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 8, Mi...|[{token, 0, 8, Mi...|[{token, 0, 8, Mi...|[{token, 0, 8, mi...|[{pos, 0, 8, NNP,...|
|Deutsche Bank rep...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 7, De...|[{token, 0, 7, De...|[{token, 0, 7, De...|[{token, 0, 7, de...|[{pos, 0, 7, NNP,...|
|Iranian "morality...|[{document, 0, 29...|[{document, 0, 14...|[{token, 0, 6, Ir...|[{token, 0, 6, Ir...|[{token, 0, 6, Ir...|[{token, 0, 6, ir...|[{

In [0]:
df_annotation.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true

In [0]:
d_pos = df_annotation.select('text','pos.metadata','pos.result')
d_pos.limit(10).toDF
d_pos.show(10)

+--------------------+--------------------+--------------------+
|                text|            metadata|              result|
+--------------------+--------------------+--------------------+
|Microsoft Corp sa...|[{word -> Microso...|[NNP, NNP, VBD, P...|
|Deutsche Bank rep...|[{word -> Deutsch...|[NNP, NNP, RB, VB...|
|Iranian "morality...|[{word -> Iranian...|[JJ, '', NN, NN, ...|
|Trump administrat...|[{word -> Trump},...|[NNP, NN, POS, VB...|
|NASA Happily Repo...|[{word -> NASA}, ...|[NNP, NNP, NNS, D...|
|President Vladimi...|[{word -> Preside...|[NNP, NNP, NNP, V...|
|33 students at 2 ...|[{word -> 33}, {w...|[CD, NNS, IN, CD,...|
|Giant tortoises b...|[{word -> Giant},...|[NNP, NNS, VBN, T...|
|The use of mariju...|[{word -> The}, {...|[DT, NN, IN, NN, ...|
|Three Tory MPs jo...|[{word -> Three},...|[CD, NNP, NNP, VB...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [0]:
from pyspark.sql import functions as f
df_pos = df_annotation.select(f.explode("pos").alias ("pos"))

In [0]:
df_pos.printSchema()
df_pos.toPandas()

root
 |-- pos: struct (nullable = true)
 |    |-- annotatorType: string (nullable = true)
 |    |-- begin: integer (nullable = false)
 |    |-- end: integer (nullable = false)
 |    |-- result: string (nullable = true)
 |    |-- metadata: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |    |-- embeddings: array (nullable = true)
 |    |    |-- element: float (containsNull = false)



,pos
0,"{'annotatorType': 'pos', 'begin': 0, 'end': 8,..."
1,"{'annotatorType': 'pos', 'begin': 10, 'end': 1..."
2,"{'annotatorType': 'pos', 'begin': 15, 'end': 1..."
3,"{'annotatorType': 'pos', 'begin': 20, 'end': 2..."
4,"{'annotatorType': 'pos', 'begin': 23, 'end': 2..."
...,...
1742,"{'annotatorType': 'pos', 'begin': 63, 'end': 7..."
1743,"{'annotatorType': 'pos', 'begin': 76, 'end': 8..."
1744,"{'annotatorType': 'pos', 'begin': 89, 'end': 9..."
1745,"{'annotatorType': 'pos', 'begin': 96, 'end': 9..."


In [0]:
nnpfilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
df_nnp = df_pos.where(nnpfilter)


Out[24]: DataFrame[pos: struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>]

In [0]:
df_nnp.show(truncate = False)

+----------------------------------------------+
|pos                                           |
+----------------------------------------------+
|{pos, 0, 8, NNP, {word -> Microsoft}, []}     |
|{pos, 10, 13, NNP, {word -> Corp}, []}        |
|{pos, 126, 131, NNP, {word -> Europe}, []}    |
|{pos, 0, 7, NNP, {word -> Deutsche}, []}      |
|{pos, 9, 12, NNP, {word -> Bank}, []}         |
|{pos, 81, 85, NNP, {word -> Trump}, []}       |
|{pos, 87, 98, NNP, {word -> Organization}, []}|
|{pos, 175, 182, NNP, {word -> Tehran's}, []}  |
|{pos, 197, 202, NNP, {word -> Narmak}, []}    |
|{pos, 221, 226, NNP, {word -> Friday}, []}    |
|{pos, 0, 4, NNP, {word -> Trump}, []}         |
|{pos, 30, 34, NNP, {word -> Saudi}, []}       |
|{pos, 88, 92, NNP, {word -> Jared}, []}       |
|{pos, 94, 100, NNP, {word -> Kushner}, []}    |
|{pos, 104, 109, NNP, {word -> Senior}, []}    |
|{pos, 111, 115, NNP, {word -> Trump}, []}     |
|{pos, 198, 202, NNP, {word -> Saudi}, []}     |
|{pos, 204, 209, NNP